<a href="https://colab.research.google.com/github/zainabemam/GitHub-Most-Popular-Repos-Analysis/blob/main/GitHub_Most_Popular_Repos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-11-jdk -y
!wget -qO - https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz | tar -xz
!pip install findspark


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libice-dev libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre x11-utils
Suggested packages:
  libice-doc libsm-doc libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libice-dev libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk openjdk-11-jre
  x11-utils
0 upgraded, 14 newly installed, 0 to remove and 29 not upgraded.
Need to get 5,522 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubun

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("GitHubReposAnalysis").getOrCreate()


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("anshulmehtakaggl/top-1000-github-repositories-for-multiple-domains")

print("Path to dataset files:", path)

100%|██████████| 5.92M/5.92M [00:00<00:00, 8.73MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/anshulmehtakaggl/top-1000-github-repositories-for-multiple-domains/versions/53


In [5]:
df = spark.read.json("/root/.cache/kagglehub/datasets/anshulmehtakaggl/top-1000-github-repositories-for-multiple-domains/versions/53/*.json")



In [6]:
df.printSchema()


root
 |-- _corrupt_record: string (nullable = true)
 |-- created: string (nullable = true)
 |-- description: string (nullable = true)
 |-- forks: long (nullable = true)
 |-- full_name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- language: string (nullable = true)
 |-- open_issues: long (nullable = true)
 |-- repo_name: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- subscribers: long (nullable = true)
 |-- topics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- type: string (nullable = true)
 |-- username: string (nullable = true)



In [7]:
df.show(5)

+---------------+-------------------+--------------------+-----+--------------------+---------+--------+-----------+-----------------+-----+-----------+--------------------+------------+---------+
|_corrupt_record|            created|         description|forks|           full_name|       id|language|open_issues|        repo_name|stars|subscribers|              topics|        type| username|
+---------------+-------------------+--------------------+-----+--------------------+---------+--------+-----------+-----------------+-----+-----------+--------------------+------------+---------+
|           null|2014-02-25 08:00:08|Apache Spark - A ...|25357|        apache/spark| 17165658|   Scala|        242|            spark|32296|       2080|[python, scala, r...|Organization|   apache|
|           null|2017-08-09 19:39:59|Distributed train...| 2027|     horovod/horovod| 99846383|  Python|        298|          horovod|12219|        334|[tensorflow, uber...|Organization|  horovod|
|           nul

In [8]:
# "programming_lang"
from pyspark.sql.functions import col, count

programming_lang_df = df.groupBy("language").agg(count("*").alias("repo_count"))
programming_lang_df = programming_lang_df.filter(col("language").isNotNull())
programming_lang_df.show()


+----------------+----------+
|        language|repo_count|
+----------------+----------+
|              C#|       336|
|            Less|         5|
|           Terra|         1|
|            Hack|         1|
|Rich Text Format|         3|
|        Makefile|        36|
|        Starlark|         6|
|             VBA|         3|
|            Cuda|        12|
|        Markdown|         3|
|             Arc|         1|
|             SMT|         1|
|            LLVM|         2|
|        ReScript|         5|
|           Metal|         1|
|            Haxe|         4|
|            Rust|       165|
|      JavaScript|      5293|
|            SCSS|        31|
|      Emacs Lisp|        18|
+----------------+----------+
only showing top 20 rows



In [10]:
# "organizations_stars"
from pyspark.sql.functions import sum

organizations_stars_df = df.filter(df.type == "Organization") \
    .groupBy("username").agg(sum("stars").alias("total_stars"))
organizations_stars_df.show()


+----------------+-----------+
|        username|total_stars|
+----------------+-----------+
|            didi|       4914|
|        deepmipt|      16875|
|        blei-lab|      14079|
|          GPflow|       4786|
|        catchorg|      14637|
|AtomicGameEngine|       5249|
|    CryptoSignal|       3977|
|        web3labs|        308|
|         blossom|        117|
|   reacttips-dev|       1039|
|        twosigma|       6150|
|     intel-spark|         91|
|            RXNT|        223|
| ml-in-barcelona|        117|
|         ngs-doo|       2334|
|   dotanuki-labs|        425|
|           rotki|       1615|
|         easzlab|      15671|
|       theonedev|      13727|
| opensourceteams|        190|
+----------------+-----------+
only showing top 20 rows



In [11]:
# "search_terms_relevance"

from pyspark.sql.functions import expr

relevance_formula = "1.5 * forks + 1.32 * subscribers + 1.04 * stars"
search_terms_df = df.withColumn("relevance_score", expr(relevance_formula)) \
                    .select(col("repo_name"), col("relevance_score"))
search_terms_df.show()


+--------------------+---------------+
|           repo_name|relevance_score|
+--------------------+---------------+
|               spark|       74368.94|
|             horovod|       16189.14|
|      SparkInternals|        8441.54|
|               delta|        6056.22|
|   TensorFlowOnSpark|        5745.24|
|              koalas|        4011.34|
|     spark-jobserver|        4671.26|
|       analytics-zoo|        3787.96|
|            ballista|        2692.12|
|               deequ|        2920.22|
|       TransmogrifAI|        2938.76|
|                vega|        2530.74|
| spark-deep-learning|        2928.80|
|spark-on-k8s-oper...|        3460.54|
|                oryx|        2768.32|
|               spark|        2329.08|
|        docker-spark|        2794.42|
|          elassandra|        2109.80|
|  spark-py-notebooks|        2966.74|
|          carbondata|        2577.14|
+--------------------+---------------+
only showing top 20 rows



In [12]:
import sqlite3

db_path = "github_repos.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [13]:
programming_lang_df_pandas = programming_lang_df.toPandas()
programming_lang_df_pandas.to_sql("programming_lang", conn, if_exists="replace", index=False)


135

In [14]:
organizations_stars_df_pandas = organizations_stars_df.toPandas()
organizations_stars_df_pandas.to_sql("organizations_stars", conn, if_exists="replace", index=False)


5627

In [16]:
search_terms_df_pandas = search_terms_df.toPandas()
# convert the Decimal objects to a type that SQLite understands, like float
search_terms_df_pandas['relevance_score'] = search_terms_df_pandas['relevance_score'].astype(float)
search_terms_df_pandas.to_sql("search_terms_relevance", conn, if_exists="replace", index=False)


27281

In [17]:
query = "SELECT * FROM programming_lang LIMIT 5;"
cursor.execute(query)
print(cursor.fetchall())


[('C#', 336), ('Less', 5), ('Terra', 1), ('Hack', 1), ('Rich Text Format', 3)]


In [18]:
conn.close()
